**Largest Lyapunov method**

The Largest Lyapunov Exponent (LLE) method is an important tool for analyzing dynamical systems, particularly for determining the presence and degree of chaos. The calculation involves creating a time series of chunks, then computing a distance matrix to measure how these chunks diverge over time. This process is part of the broader LLE calculation.

In [1]:
import numpy as np
import math
import itertools
import matplotlib.pyplot as plt
import warnings
!pip install DBSCAN1D

from tqdm import tqdm
from scipy.spatial import distance
from sklearn.cluster import DBSCAN
from dbscan1d.core import DBSCAN1D

In [2]:
!wget https://raw.githubusercontent.com/mlkyai/TS/main/lorenz.txt

--2023-12-23 21:15:08--  https://raw.githubusercontent.com/mlkyai/TS/main/lorenz.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2500025 (2.4M) [text/plain]
Saving to: ‘lorenz.txt.2’

lorenz.txt.2        100%[===================>]   2.38M  --.-KB/s    in 0.07s   

2023-12-23 21:15:08 (36.3 MB/s) - ‘lorenz.txt.2’ saved [2500025/2500025]



In [3]:
!ls

lorenz.txt  lorenz.txt.1  lorenz.txt.2	sample_data


In [4]:
def chunks(time_series, s):

    # number of all possible chunks
    num_chunks = len(time_series) - s + 1
    chunks = np.zeros((num_chunks, s))
    for i in range(num_chunks):
      chunks[i] = time_series[i:i+s]
    return chunks

In [9]:
def calculate_distance_matrix(chunks_list, num_chunks):
    # distance matrix
    distance_matrix = np.zeros((num_chunks, num_chunks))

    # pairwise distances
    for i in range(num_chunks):
        distances = np.linalg.norm(chunks_list[i] - chunks_list, axis=1)
        distance_matrix[i, :] = distances

    return distance_matrix

In [16]:
def calculate_LLE(time_series, start, finish, s, k, epsilon):
    # TS chunks
    chunks_list = chunks(time_series[start:finish + k], s)
    # num_chunks = finish - start - s + 1
    num_chunks = len(chunks_list)

    # Distance matrix
    distance_matrix = calculate_distance_matrix(chunks_list, num_chunks)

    # Indices of neighbors within epsilon distance
    keep_index = []
    for i in range(num_chunks - k):
        neighbors = [j for j in range(num_chunks) if distance_matrix[i, j] < epsilon and abs(j - i) > 2 * s]
        keep_index.append(neighbors)

    # Largest Lyapunov Exponent
    LLE = sum(np.log(distance_matrix[i + k, n + k] / distance_matrix[i, n])
              for i in range(num_chunks - k) for n in keep_index[i] if n < num_chunks - k)
    LLE /= k * (num_chunks - k)

    return LLE

In [17]:
# File path for the data file
file_path = "lorenz.txt"

# convertion of file to an array
TS = np.loadtxt(file_path)

print(calculate_LLE(TS, 1, 1000, s=10, k=10, epsilon= 0.5))

9.45310588450371
